In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm
gamini_key=os.getenv("GAMINI_API_KEY")

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter                          

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
url=[
    "https://langchain-ai.github.io/langgraph/concepts/why-langgraph/",
    "https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/",
    "https://langchain-ai.github.io/langgraph/tutorials/get-started/4-human-in-the-loop/"
]

docs=[WebBaseLoader(url).load() for url in url]

In [7]:
docs

[[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/why-langgraph/', 'title': 'Overview', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\nOverview\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nLangGraph Platform docs have moved! Find the LangGraph Platform docs at the new LangChain Docs site.\n\n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            LangGraph\n          \n\n\n\n            \n              Overview\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n  \n    \n  \n  Get started\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Guides\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Reference\n\n        \n\n\n\n       

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Setup Gemini embeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key= gamini_key # Or set GOOGLE_API_KEY environment variable
)

In [9]:
docs_list=[item for sublist in docs for item in sublist]
text_spliter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=50)
doc_split=text_spliter.split_documents(docs_list)

vectorstore=FAISS.from_documents(
    documents=doc_split,
    embedding=gemini_embeddings
)

retriver=vectorstore.as_retriever()

In [10]:
retriver.invoke("what is langgraph")

[Document(id='07935e8c-c8e9-4f99-8a73-97e9daef78c2', metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/', 'title': '2. Add tools', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='LangGraph works in conjunction with LangSmith, another tool by LangChain, to provide an'),
 Document(id='c8383a74-5c58-4a26-8844-bbedaef62c97', metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/', 'title': '2. Add tools', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to'),
 Document(id='dde2c1e8-2cae-4cc9-bac0-b07cd15adefa', metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/why-langgraph/', 'title': 'Overview', 'description': 'Build reliable, stateful AI systems, without g

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever=create_retriever_tool(
    retriver,
    "vector",
    "search and run information about langgrah"
)

In [14]:
retriever

Tool(name='vector', description='search and run information about langgrah', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000028F3DD34720>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028F77E45520>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000028F3DD34B80>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028F77E45520>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content

In [15]:
tools=[retriever]

In [ ]:
from langgraph.graph import StateGraph , START ,END
from langgraph.prebuilt import ToolNode,tools_condition

graph=StateGraph(AgentState)
graph.add_node('agent',agent)
retriever=ToolNode([retriever_tool])
graph.add
